### Flexible Paleoclimate Age-Depth Models Using an Autoregressive Gamma Process

In [ ]:
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt
from twalk import  pytwalk
import pandas as pd
from numba import njit

In [3]:
# Parameters
K = 21 # Number of equally spaced sections
delta_c = 5 # space between adjacent sections

# accumulation rate parameters
acc_shape = 1.5
acc_mean = 20 # mean of accumulation rate (yr/cm unit)

# memory strength parameters
mem_strength = 10
mem_mean = 0.5 # mean of memory strength [0,1]. 0 means no memory, 1 means full memory

# Student-t parameters
a = 3
b = 4

In [4]:
# Setting parameters for Gamma distribution
# mean of gamma dist = alpha * beta
a_alpha = acc_shape
a_beta = acc_shape / acc_mean

# Setting parameters for Beta distribution
# mean of beta dist = aw / (aw + bw)
aw = mem_strength
bw = (aw - mem_mean * aw) / mem_mean

ds = 1

In [ ]:
# load data
dataset = pd.read_csv('MyLab.csv')
calib_curve = pd.read_csv('calib_curves/intcal20.csv')

y = dataset['age'].values
depths = dataset['depth'].values
error = dataset['error'].values

M = len(y)

# equally spaced sections
c = np.arange(1.5, (K + 1) * delta_c, delta_c)
y

In [6]:
# calibration curve (IntCal20)
# returns (age, sigma)
cal_bp = calib_curve['CAL BP'][::-1]
c14_age = calib_curve['14C age'][::-1]
sigma = calib_curve['Sigma'][::-1]

def mu(theta):
    return np.interp(theta, cal_bp, c14_age), np.interp(theta, cal_bp, sigma)

# precomputing the interpolated values for all possible calender years
calibrated_age, calibrated_std = mu(np.arange(cal_bp.min(), cal_bp.max() + 1))

In [7]:
# semi-parametric model to calculate calendar years for a given depth
# returns calendar year
# expects partitioned x values
@njit
def G(theta, x, d):
    i = int(d / delta_c)

    sum_part = 0.0
    for j in range(i + 1):
        sum_part += x[j] * delta_c

    remainder = x[i + 1] * (d - c[i])
    return theta + sum_part + remainder

@njit
def compute_G_values(theta, x, depths):
    n = len(depths)
    G_values = np.empty(n, dtype=np.float64)
    for idx in range(n):
        G_values[idx] = G(theta, x, depths[idx])
        
    return G_values

In [ ]:
# prior for theta. theta ~ N(y[0], error[0]) (Christen Fox 2010).
def theta_prior(theta):
    return norm.pdf(theta, loc=y[0], scale=error[0])

In [9]:
# "enery" function U(theta, x, w | y) = -log (f(theta, x, w | y))
def prioracU(i, al):
    return (1.0 - a_alpha) * np.log(al) + a_beta * al

def energy(x):
    theta = x[0]
    w = x[K + 1]

    G_values = compute_G_values(theta, x[1 : K + 1], depths)
    mu_values = calibrated_age[G_values.astype(int)]
    std_values = calibrated_std[G_values.astype(int)]

    term_1 = ((a + 0.5) * np.log(b + ((y - mu_values)**2) / (2 * (error**2 + std_values**2)))).sum()


    term_2 = -np.log(theta)

    # Term 3: Prior for w
    term_3 = (1 - aw) * np.log(w) / delta_c + (1 - bw) * np.log(1 - w**(ds / delta_c))

    # Term 4: Prior for x
    term_4 = prioracU(0, x[K])
    for k in range(1, K):
        term_4 += prioracU(0, (x[k] - w * x[k + 1]) / (1 - w))

    # Total energy
    return (term_1 + term_2 + term_3 + term_4)


In [10]:
def support(x):
    w = x[-1]

    if w <= 0 or w >= 1:
        return False
    
    if x[0] <= 0:
        return False

    if x[K] <= 0:
        return False
        
    for i in range(K - 1, 0, -1):
        ak = (x[i] - w * x[i + 1]) / (1 - w)
        if ak < 0:
            return False
        
    return True

In [ ]:
x = np.zeros(K + 2, dtype=np.float64) # x[0] = theta and x[-1] = w
x_dash = np.zeros(K + 2, dtype=np.float64)
# set initial values
# theta
x[0] = 4105
x_dash[0] = 4212

# setting initial values for w
x[-1] = 0.04
x_dash[-1] = 0.01

# setting initial values for x
x[K] = np.random.gamma(a_alpha, a_beta / (1 - x[-1]))
x_dash[K] = np.random.gamma(a_alpha, a_beta / (1 - x_dash[-1]))

for i in range(K - 1, 0, -1):
    x[i] = x[-1] * x[i + 1] + np.random.gamma(a_alpha, a_beta / (1 - x[-1]))
    x_dash[i] = x_dash[-1] * x_dash[i + 1] + np.random.gamma(a_alpha, a_beta / (1 - x_dash[-1]))

energy(x)

In [ ]:
thin = 30 * (K + 2)
burn_in = 20_000 * (K + 2)
iterations = 4000

T = iterations * thin + burn_in

mcmc = pytwalk.pytwalk(n = K + 2, U = energy, Supp = support)
mcmc.Run(T=T, x0=x, xp0=x_dash, thin=thin)

In [ ]:
mcmc.SavetwalkOutput('output/output.csv', burn_in=burn_in)

In [14]:
theta = mcmc.Output[1000:, 0]
x = mcmc.Output[1000:, 1:K + 1]
w = mcmc.Output[1000:, -2]
objective = mcmc.Output[1000:, -1]

def G(theta, x, d):
    # Find which interval d falls into (0-based)
    i = int(d / delta_c)

    memory = sum([x[j] * delta_c for j in range(i + 1)])

    return theta + memory + x[i + 1] * (d - c[i])

In [ ]:
# make a trace plot of objective
plt.plot(objective)
plt.xlabel('Iteration')
plt.ylabel('Objective')
plt.title('Objective Trace Plot')
plt.show()

In [ ]:
# plot histogram of theta
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(24, 6))
ax1.hist(theta, bins=50)
ax1.set_title('Posterior distribution of Theta')
ax2.hist(w**(ds / delta_c), bins=50)
ax2.set_title('Posterior distribution of R')
ax3.hist(x[:,0], bins=50)
ax3.set_title('Posterior distribution of X')
plt.show()

In [17]:
test_depths = np.arange(depths.min(), depths.max() + 1, 1)
results = np.array([np.array([G(theta[i], x[i], d) for i in range(len(theta))]) for d in test_depths])

In [ ]:
result_frame = pd.DataFrame(columns=['depth', 'min', 'max', 'median', 'mean'])

for i in range(len(results)):
    result_frame.loc[i] = [test_depths[i], round(np.min(results[i]), 2), round(np.max(results[i]), 2), round(np.median(results[i]), 2), round(np.mean(results[i]), 2)]

result_frame